In [10]:
pip install requests beautifulsoup4 selenium webdriver-manager


In [21]:
# ✅ Install Chrome & ChromeDriver
!apt-get update -y
!apt-get install -y wget unzip curl
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb
!google-chrome --version

# Get Chrome major version
!CHROME_VERSION=$(google-chrome --version | awk '{print $3}' | cut -d '.' -f 1) && \
DRIVER_VERSION=$(curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE_${CHROME_VERSION}) && \
wget -q https://chromedriver.storage.googleapis.com/${DRIVER_VERSION}/chromedriver_linux64.zip && \
unzip -o chromedriver_linux64.zip -d /usr/local/bin/ && \
chmod +x /usr/local/bin/chromedriver

# ✅ Install required Python packages
!pip install -q selenium beautifulsoup4


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,224 B]
Fetched 3,049 B in 2s (1,582 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repos

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

def scrape_credly_badges(profile_url):
    # ✅ Setup Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(profile_url)

    # ✅ Wait for badge containers
    WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="desktop-badge-card"]'))
    )

    # ✅ Scroll to load all badges
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # ✅ Select all badge cards
    badge_cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="desktop-badge-card"]')

    badges = []
    for card in badge_cards:
        name = card.find_element(By.CSS_SELECTOR, 'span[data-testid="Typography"]').text.strip()

        # ✅ Extract URL directly from `href` attribute of the <div>
        relative_url = card.get_attribute("href")
        full_url = f"https://www.credly.com{relative_url}" if relative_url else None

        badges.append({"name": name, "url": full_url})

    driver.quit()

    # ✅ Save CSV
    with open("credly_badges.csv", "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "url"], delimiter="\t")
        writer.writeheader()
        writer.writerows(badges)

    return badges


# ✅ Run scraper
url = "Your credly LINK"
badges = scrape_credly_badges(url)

# ✅ Print results
print(f"✅ Total Badges Found: {len(badges)}\n")
for b in badges:
    print(f"name\t\"{b['name']}\"")
    print(f"url\t\"{b['url']}\"\n")


✅ Total Badges Found: 73

name	"Containers & Kubernetes Essentials"
url	"https://www.credly.com/badges/d84a7abb-4d49-49d8-bece-58e956a6233e"

name	"DevOps, Cloud, and Agile Foundations Specialization"
url	"https://www.credly.com/badges/6f769cb8-edb1-4bc5-b264-8e9fe76bb6cb"

name	"Introduction to Agile Development and Scrum"
url	"https://www.credly.com/badges/d82ecfa4-f223-4506-97ff-80ab7ed1e0e0"

name	"Introduction to Cloud Computing"
url	"https://www.credly.com/badges/fa782fb4-2c62-4051-8b33-f23aeaea1e55"

name	"DevOps Essentials"
url	"https://www.credly.com/badges/8b807dc6-0fe2-4651-9d8b-3ecabae47bff"

name	"Generative AI Essentials"
url	"https://www.credly.com/badges/088c28ae-9e57-427e-8a3e-adb439d59a40"

name	"Generative AI: Prompt Engineering"
url	"https://www.credly.com/badges/da5b7c6a-f231-42fc-aec4-59c136739467"

name	"Statistics Essentials Using Excel"
url	"https://www.credly.com/badges/e6d30d6c-e66e-47ab-872f-16650741315c"

name	"Generative AI Essentials for Data Analytics"
u